In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import datetime
import backtrader as bt


class RSIStrategy(bt.Strategy):

    params = (
        ("verbose", True),
        ("maperiod", None),
        ("quantity", None),
        ("upper", 70),  # RSI Upper threshold
        ("lower", 30),  # RSI Lower threshold
        ("stopLoss", 0.0)  # Stop-Loss percentage
    )

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.order_stopLoss = None
        self.buyprice = None
        self.buycomm = None
        self.amount = None

        # Add a Moving Average Simple indicator
        self.rsi = bt.indicators.RSI_SMA(self.datas[0], period=self.params.maperiod, safediv=True)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm

                if self.params.verbose:
                    print(
                        "BOUGHT at price: {:.2f} {}".format(
                            order.executed.price, bt.num2date(order.executed.dt)
                        )
                    )
                if self.params.stopLoss > 0:
                    stop_price = order.executed.price * (1 - self.params.stopLoss)
                    self.order_stopLoss = self.sell(
                        exectype=bt.Order.Stop, price=stop_price
                    )
                    if self.params.verbose:
                        print("STOP at price: {:.2f}".format(stop_price))
                else:
                    # Trailing stop specified percentage under executed price
                    self.order_stopLoss = self.sell(
                        exectype=bt.Order.StopTrail,
                        trailpercent=0 - self.params.stopLoss,
                    )
                    self.order_stopLoss.addinfo(ordername="STOPLONG")
                    if self.params.verbose:
                        print("STOP TRAILING")

            else:
                if not self.position:  # Elvis has left the market
                    self.broker.cancel(self.order_stopLoss)
                    self.order_stopLoss = None
                    if self.params.verbose:
                        print(
                            "SOLD at price: {:.2f} cost: {:.2f} {}".format(
                                order.executed.price,
                                order.executed.value,
                                order.executed.comm,
                                bt.num2date(order.executed.dt),
                            )
                        )

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        if self.params.verbose:
            print("PROFIT, GROSS %.2f, NET %.2f" % (trade.pnl, trade.pnlcomm))
            print("__________________________________________")

    def next(self):

        # Check if an order is pending ... if yes, we cannot send a 2nd one

        if self.order:
            return
        # Check if we are in the market
        if not self.position:

            # Not yet ... We _might_ BUY if ...
            if self.rsi < self.params.lower:

                # Keep track of the created order to avoid a 2nd order
                self.amount = (
                    self.broker.getvalue() * self.params.quantity
                ) / self.dataclose[0]
                self.order = self.buy()

        else:
            # Already in the market ... We _might_ SELL if ...
            if self.rsi > self.params.upper:

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell


# ___________ Strategy Class Finished __________ #

import re


def timeFrame(datapath):
    """
    Select the write compression and timeframe
    """
    sepdatapath = re.split(" |_", datapath)
    tf = sepdatapath[1]

    tfs = [
        "1min",
        "3min",
        "5min",
        "15min",
        "30min",
        "1hour",
        "2hour",
        "4hour",
        "6hour",
        "8hour",
        "12hour",
        "1-day",
        "3-day",
        "1-week",
        "1-month"
    ]

    # Set compression and timeframe
    minutes_hours = range(len(tfs[:-4]))
    if 'min' or 'hour' in tf:
        for time in minutes_hours:
            if tf == tfs[time]:
                if 'min' in tf:
                    comptime = int(re.sub('min', '', tf))
                    compression = comptime
                elif 'hour' in tf:
                    comptime = int(re.sub('hour', '', tf))
                    compression = comptime * 60
            timeframe = bt.TimeFrame.Minutes
    elif 'day' in tf:
        if tf == '1-day':
            compression = 1
        elif tf == '3-day':
            compression = 3
        timeframe = bt.TimeFrame.Days
    elif tf == '1-week':
        compression = 1
        timeframe = bt.TimeFrame.Weeks
    elif tf == '1-month':
        compression = 1
        timeframe = bt.TimeFrame.Months
    else:
        print("Dataframe not recognized")
        exit()
    print(compression,timeframe)
    print(type(compression), type(timeframe))
    return compression, timeframe


def getWinLoss(analyzer):
    return analyzer.won.total, analyzer.lost.total, analyzer.pnl.net.total


def getSQN(analyzer):
    return round(analyzer.sqn, 2)


def runbacktest(
    datapath,
    start,
    end,
    period,
    strategy,
    upper=70,
    lower=30,
    commission_val=None,
    portfolio=10000.0,
    stake_val=1,
    quantity=0.01,
    stopLoss=0.0,
    plt=False,
):

    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=stake_val)  # Multiply the stake by X

    cerebro.broker.setcash(portfolio)

    if commission_val:
        cerebro.broker.setcommission(commission=commission_val / 100)

    # Add a strategy
    # if strategy == 'SMA':
    #     cerebro.addstrategy(SMAStrategy, maperiod = period, quantity = quantity, stopLoss = stopLoss, upper = upper, lower = lower)
    if strategy == "RSI":
        cerebro.addstrategy(
            RSIStrategy,
            maperiod=period,
            quantity=quantity,
            stopLoss=stopLoss,
            upper=upper,
            lower=lower,
        )
    else:
        print("No Strat")
        exit()

    compression, timeframe = timeFrame(datapath)

    data = bt.feeds.GenericCSVData(
        dataname=datapath,
        dtformat=2,
        comperssion=compression,
        timeframe=timeframe,
        fromdate=datetime.datetime.strptime(start, "%d-%b-%Y"),
        todate=datetime.datetime.strptime(end, "%d-%b-%Y"),
        reverse=False,
    )

    # Create a Data Feed to Cerebro
    cerebro.adddata(data)

    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
    cerebro.addanalyzer(bt.analyzers.SQN, _name="sqn")

    try:
        if True:
            strat = cerebro.run()
            stratexe = strat[0]

            try:
                totalwin, totalloss, pnl_net = getWinLoss(
                    stratexe.analyzers.ta.get_analysis()
                )
            except KeyError:
                totalwin, totalloss, pnl_net = 0, 0, 0

            sqn = getSQN(stratexe.analyzers.sqn.get_analysis())

            if plt:
                cerebro.plot()

                return cerebro.broker.getvalue(), totalwin, totalloss, pnl_net, sqn

    except Exception as e:
        print("Error:", str(e))
        return 0, 0, 0, 0, 0


TypeError: module() takes at most 2 arguments (3 given)